In [4]:
import os
root = os.path.dirname(os.path.abspath("."))
REPO_ROOT = os.path.join(root, "..")

import sys
sys.path.append(REPO_ROOT)

DATA_DIR = os.path.join(REPO_ROOT, "data", "data_raw", "CruParamer")

In [19]:
train_1 = os.path.join(DATA_DIR, "Datacon2019-Malicious-Code-DataSet-Stage1", "train", "black")
train_0 = os.path.join(DATA_DIR, "Datacon2019-Malicious-Code-DataSet-Stage1", "train", "white")

sample = os.path.join(DATA_DIR, [x for x in os.listdir(DATA_DIR) if x.endswith("xml")][0])

In [20]:
from lxml import etree

def parseXML(xmlFile):
    """
    Parse the xml
    """
    parser = etree.XMLParser(recover=True)
    with open(xmlFile) as fobj:
        xml = fobj.read()
    print(len(xml))
    print(xml[0:10])
    root = etree.fromstring(xml, parser=parser)
    return root
    # for appt in root.getchildren():
    #     for elem in appt.getchildren():
    #         if not elem.text:
    #             text = "None"
    #         else:
    #             text = elem.text
    #         print(elem.tag + " => " + text)

xmlsample = parseXML(sample)

81175
<report re


In [23]:
for i, element in enumerate(xmlsample.iter()):
    print(element.tag, element.attrib)
    if i == 10:
        break

report {'report_id': 'EAE38ADA', 'report_version': '2'}
file_list {'file_error': '0', 'file_name': '00a92821ae644a257e7c2bb0dc1a738e05bb377d46a574ea745e8643362675f6.exe', 'file_uid': '780048'}
file {}
start_boot {}
field {'value': 'start_boot'}
action_list {}
action {'api_name': 'AnalyzeStart', 'call_name': '', 'call_pid': '0', 'call_time': '18:45:37.000', 'ret_value': '0'}
apiArg_list {'count': '0'}
exInfo_list {'count': '0'}
action {'api_name': 'Fake_BeCreatedEx', 'call_name': '1.exe', 'call_pid': '396', 'call_time': '18:45:40.005', 'ret_value': '0'}
apiArg_list {'count': '0'}


Actual XML file:

```
<report report_id="EAE38ADA" report_version="2">
    <file_list file_error="0" file_name="00a92821ae644a257e7c2bb0dc1a738e05bb377d46a574ea745e8643362675f6.exe" file_uid="780048">
        <file>
            <start_boot>
                <field value="start_boot" />
                <action_list>
                    <action api_name="AnalyzeStart" call_name="" call_pid="0" call_time="18:45:37.000" ret_value="0">
                        <apiArg_list count="0" />
                        <exInfo_list count="0" />
```

In [18]:
print([x for x in dir(xmlsample) if "__" not in x])

['_init', 'addnext', 'addprevious', 'append', 'attrib', 'base', 'clear', 'cssselect', 'extend', 'find', 'findall', 'findtext', 'get', 'getchildren', 'getiterator', 'getnext', 'getparent', 'getprevious', 'getroottree', 'index', 'insert', 'items', 'iter', 'iterancestors', 'iterchildren', 'iterdescendants', 'iterfind', 'itersiblings', 'itertext', 'keys', 'makeelement', 'nsmap', 'prefix', 'remove', 'replace', 'set', 'sourceline', 'tag', 'tail', 'text', 'values', 'xpath']


In [68]:
xmlsample.getchildren()

[<Element file_list at 0x236462a9440>]

In [46]:
etree.dump(xmlsample)

<report report_id="EAE38ADA" report_version="2">
    <file_list file_error="0" file_name="00a92821ae644a257e7c2bb0dc1a738e05bb377d46a574ea745e8643362675f6.exe" file_uid="780048">
        <file>
            <start_boot>
                <field value="start_boot"/>
                <action_list>
                    <action api_name="AnalyzeStart" call_name="" call_pid="0" call_time="18:45:37.000" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="Fake_BeCreatedEx" call_name="1.exe" call_pid="396" call_time="18:45:40.005" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="4">
                            <exInfo value="SyStem.exe"/>
                            <exInfo value="212"/>
                            <exInfo value="&quot;C:\program\1.exe&quot; "/>
                            <exInfo value="direct"/>

In [127]:
def parse_cruparamer_xmlsample(xmlsample):
    parsed = []
    for action in xmlsample.xpath("//report//file_list//file//start_boot//action_list")[0].getchildren():
        name = action.get("api_name").lower()
        args, exinfo = action.getchildren()
        args = " ".join([x.values()[0].lower() for x in args.getchildren()])
        exinfo = " ".join([x.values()[0].lower() for x in exinfo.getchildren()])
        parsed.append(f"{name} {args} {exinfo}".strip())
    return parsed

parse_cruparamer_xmlsample(xmlsample)

['analyzestart',
 'fake_becreatedex  system.exe 212 "c:\\program\\1.exe"  direct',
 'trytoanalyze  c:\\program\\1.exe',
 'unpackself',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\advapi32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\rpcrt4.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\secur32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\comctl32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\gdi32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\user32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\ole32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\msvcrt.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\oleaut32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\shell32.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\shlwapi.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\urlmon.dll',
 'loadlibraryexw 0 0 0 c:\\windows\\system32\\version.dll',
 'ntqueryattributesfile 12f5f4 12f5cc c:\\program\\winspool.drv',
 'loadlibraryexw 0 0 0 c

In [101]:
action = xmlsample.xpath("//report//file_list//file//start_boot//action_list")[0].getchildren()[17]
print(action.get("api_name"))
args, exinfo = action.getchildren()
print([x.values()[0] for x in args.getchildren()])
[x.values()[0] for x in exinfo.getchildren()]

NtQueryAttributesFile
['12f5f4', '12f5cc']


['C:\\program\\winspool.drv']

In [107]:
test = """<report report_id="EAE38ADA" report_version="2">
    <file_list file_error="0" file_name="00a92821ae644a257e7c2bb0dc1a738e05bb377d46a574ea745e8643362675f6.exe" file_uid="780048">
        <file>
            <start_boot>
                <field value="start_boot"/>
                <action_list>
                    <action api_name="AnalyzeStart" call_name="" call_pid="0" call_time="18:45:37.000" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="Fake_BeCreatedEx" call_name="1.exe" call_pid="396" call_time="18:45:40.005" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="4">
                            <exInfo value="SyStem.exe"/>
                            <exInfo value="212"/>
                            <exInfo value="&quot;C:\\program\\1.exe&quot; "/>
                            <exInfo value="direct"/>
                        </exInfo_list>
                    </action>
                    <action api_name="TryToAnalyze" call_name="1.exe" call_pid="396" call_time="18:45:40.006" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.exe"/>
                        </exInfo_list>
                    </action>
                    <action api_name="UnpackSelf" call_name="1.exe" call_pid="396" call_time="18:45:40.007" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.020" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\advapi32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.022" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\rpcrt4.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.024" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\secur32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.026" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\comctl32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.028" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\gdi32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.030" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\user32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.032" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\ole32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.034" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\msvcrt.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.036" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\oleaut32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.038" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\shell32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.040" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\shlwapi.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.042" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\urlmon.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.044" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\version.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.046" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f5f4"/>
                            <apiArg value="12f5cc"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\winspool.drv"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.052" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\winspool.drv"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtReadFile" call_name="1.exe" call_pid="396" call_time="18:45:40.055" ret_value="0">
                        <apiArg_list count="9">
                            <apiArg value="1c"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="12fae8"/>
                            <apiArg value="12fb00"/>
                            <apiArg value="4"/>
                            <apiArg value="12fb0c"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.exe"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.073" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\imm32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.079" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12eae8"/>
                            <apiArg value="12eac0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\LPK.DLL"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.082" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\lpk.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.084" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12e7b8"/>
                            <apiArg value="12e790"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\USP10.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.087" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\usp10.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.123" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12e7ec"/>
                            <apiArg value="12e7c4"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.exe.Local\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.128" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\WinSxS\\x86_Microsoft.Windows.Common-Controls_6595b64144ccf1df_6.0.2600.5512_x-ww_35d4ce83\\comctl32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtReadFile" call_name="1.exe" call_pid="396" call_time="18:45:40.135" ret_value="40">
                        <apiArg_list count="16">
                            <apiArg value="48"/>
                            <apiArg value="0"/>
                            <apiArg value="2"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="143e58"/>
                            <apiArg value="12e61a"/>
                            <apiArg value="2080000"/>
                            <apiArg value="12e604"/>
                            <apiArg value="a0008"/>
                            <apiArg value="7c814e90"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\WindowsShell.Manifest"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.148" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12f8d8"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12f8b8"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="ZonesCounterMutex"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.149" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12f8cc"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12f8ac"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="ZonesCacheCounterMutex"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.150" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12f8cc"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12f8ac"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="ZonesLockedCacheCounterMutex"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.153" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12e51c"/>
                            <apiArg value="12e4f4"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.exe.Local\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.185" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f4a8"/>
                            <apiArg value="12f480"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.CHS"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.187" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f314"/>
                            <apiArg value="12f2ec"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.CHS.DLL"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.188" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f4a8"/>
                            <apiArg value="12f480"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.CH"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.190" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f314"/>
                            <apiArg value="12f2ec"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.CH.DLL"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.198" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\msctf.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.205" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12eec0"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12eea0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="CTF.LBES.MutexDefaultS-1-5-21-117609710-838170752-515967899-1003"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.206" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12eec0"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12eea0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="CTF.Compart.MutexDefaultS-1-5-21-117609710-838170752-515967899-1003"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.207" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12eec0"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12eea0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="CTF.Asm.MutexDefaultS-1-5-21-117609710-838170752-515967899-1003"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.208" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12eec0"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12eea0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="CTF.Layouts.MutexDefaultS-1-5-21-117609710-838170752-515967899-1003"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.209" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12eec0"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12eea0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="CTF.TMD.MutexDefaultS-1-5-21-117609710-838170752-515967899-1003"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:40.213" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12f258"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12f238"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="CTF.TimListCache.FMPDefaultS-1-5-21-117609710-838170752-515967899-1003MUTEX.DefaultS-1-5-21-117609710-838170752-515967899-1003"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtReadFile" call_name="1.exe" call_pid="396" call_time="18:45:40.225" ret_value="88">
                        <apiArg_list count="16">
                            <apiArg value="8c"/>
                            <apiArg value="0"/>
                            <apiArg value="2"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="7c80ac9f"/>
                            <apiArg value="0"/>
                            <apiArg value="143d08"/>
                            <apiArg value="12eb68"/>
                            <apiArg value="2080000"/>
                            <apiArg value="12eb40"/>
                            <apiArg value="a0008"/>
                            <apiArg value="7c814e90"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\msctfime.ime"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.246" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\msctfime.ime"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:40.266" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f740"/>
                            <apiArg value="12f718"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\olepro32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:40.269" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\olepro32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="FindFirstFileExW" call_name="1.exe" call_pid="396" call_time="18:45:41.295" ret_value="ffffffff">
                        <apiArg_list count="6">
                            <apiArg value="7ffdfc00"/>
                            <apiArg value="0"/>
                            <apiArg value="12f988"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.299" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="a2"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149290"/>
                            <apiArg value="24"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\CLSID\\{D173E10A-001D-4318-9822-8C97A8418482}"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="FindFirstFileExW" call_name="1.exe" call_pid="396" call_time="18:45:41.307" ret_value="ffffffff">
                        <apiArg_list count="6">
                            <apiArg value="7ffdfc00"/>
                            <apiArg value="0"/>
                            <apiArg value="12f9cc"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.311" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="9e"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149290"/>
                            <apiArg value="22"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\CLSID\\{D173E10A-001D-4318-9822-8C97A8418482}\\LOCALSERVER32"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.315" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="a2"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149290"/>
                            <apiArg value="24"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\1.EXTERNALNSHANDLER"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.319" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="9e"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149478"/>
                            <apiArg value="4e"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\1.EXTERNALNSHANDLER\\CLSID"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.323" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="a2"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149290"/>
                            <apiArg value="28"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\CLSID\\{D173E10A-001D-4318-9822-8C97A8418482}\\PROGID"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="FindFirstFileExW" call_name="1.exe" call_pid="396" call_time="18:45:41.331" ret_value="ffffffff">
                        <apiArg_list count="6">
                            <apiArg value="7ffdfc00"/>
                            <apiArg value="0"/>
                            <apiArg value="12f988"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.335" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="9e"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="14df90"/>
                            <apiArg value="1e"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\CLSID\\{9C453F21-396D-11D5-9734-70E252C10127}"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="FindFirstFileExW" call_name="1.exe" call_pid="396" call_time="18:45:41.343" ret_value="ffffffff">
                        <apiArg_list count="6">
                            <apiArg value="7ffdfc00"/>
                            <apiArg value="0"/>
                            <apiArg value="12f9cc"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.347" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="a2"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149290"/>
                            <apiArg value="22"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\CLSID\\{9C453F21-396D-11D5-9734-70E252C10127}\\LOCALSERVER32"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.351" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="9e"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="14df90"/>
                            <apiArg value="1e"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\1.EBOOKNSHANDLER"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.355" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="a2"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149478"/>
                            <apiArg value="4e"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\1.EBOOKNSHANDLER\\CLSID"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.359" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="9e"/>
                            <apiArg value="12fe54"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="149290"/>
                            <apiArg value="22"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\CLASSES\\CLSID\\{9C453F21-396D-11D5-9734-70E252C10127}\\PROGID"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.363" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f624"/>
                            <apiArg value="12f5fc"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\CLBCATQ.DLL"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.366" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\clbcatq.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.368" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f2f4"/>
                            <apiArg value="12f2cc"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\COMRes.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.371" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\comres.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtReadFile" call_name="1.exe" call_pid="396" call_time="18:45:41.391" ret_value="0">
                        <apiArg_list count="9">
                            <apiArg value="110"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="12f900"/>
                            <apiArg value="150118"/>
                            <apiArg value="56f8"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\Registration\\R000000000007.clb"/>
                        </exInfo_list>
                    </action>
                    <action api_name="Fake_DetectDebugger" call_name="1.exe" call_pid="396" call_time="18:45:41.417" ret_value="1">
                        <apiArg_list count="1">
                            <apiArg value="12f7fc"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="MemoryOperation"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.425" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f0b4"/>
                            <apiArg value="12f08c"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\xpsp2res.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.428" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\xpsp2res.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="GetComputerNameW" call_name="1.exe" call_pid="396" call_time="18:45:41.431" ret_value="1">
                        <apiArg_list count="2">
                            <apiArg value="77edb3ec"/>
                            <apiArg value="77edb070"/>
                        </apiArg_list>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="NtWriteFile" call_name="1.exe" call_pid="396" call_time="18:45:41.440" ret_value="0">
                        <apiArg_list count="9">
                            <apiArg value="14c"/>
                            <apiArg value="121"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="12f448"/>
                            <apiArg value="157f68"/>
                            <apiArg value="48"/>
                            <apiArg value="12f418"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="NamedPipe\\lsarpc"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtReadFile" call_name="1.exe" call_pid="396" call_time="18:45:41.441" ret_value="0">
                        <apiArg_list count="9">
                            <apiArg value="14c"/>
                            <apiArg value="121"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="144cf4"/>
                            <apiArg value="158078"/>
                            <apiArg value="400"/>
                            <apiArg value="12f41c"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="NamedPipe\\lsarpc"/>
                        </exInfo_list>
                    </action>
                    <action api_name="GetComputerNameW" call_name="1.exe" call_pid="396" call_time="18:45:41.451" ret_value="1">
                        <apiArg_list count="2">
                            <apiArg value="1586e8"/>
                            <apiArg value="12fba4"/>
                        </apiArg_list>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="Fake_DetectDebugger" call_name="1.exe" call_pid="396" call_time="18:45:41.459" ret_value="abadbeef">
                        <apiArg_list count="2">
                            <apiArg value="0"/>
                            <apiArg value="12cff50"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="sleep(60s)"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.483" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12f274"/>
                            <apiArg value="12f24c"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\uxtheme.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.486" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\uxtheme.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.539" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\shdocvw.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.541" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\crypt32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.543" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\msasn1.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.545" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\cryptui.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.547" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\netapi32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.549" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\wininet.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.551" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\wintrust.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.553" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\imagehlp.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.555" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\wldap32.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.560" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12cea8"/>
                            <apiArg value="12ce80"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.exe.Local\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.570" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12da74"/>
                            <apiArg value="12da4c"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\RichEd20.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.573" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\riched20.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.578" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12ceac"/>
                            <apiArg value="12ce84"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.exe.Local\\"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.609" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1b0"/>
                            <apiArg value="12ea98"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="12eb68"/>
                            <apiArg value="92"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\EXPLORER\\SHELL FOLDERS"/>
                            <exInfo value="Cache"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.617" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1b4"/>
                            <apiArg value="12ec30"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="15c908"/>
                            <apiArg value="aa"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS"/>
                            <exInfo value="Directory"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.618" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1b4"/>
                            <apiArg value="12ec38"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12ecac"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS"/>
                            <exInfo value="Paths"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.619" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1b8"/>
                            <apiArg value="12ec30"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="166d28"/>
                            <apiArg value="b8"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH1"/>
                            <exInfo value="CachePath"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.620" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1bc"/>
                            <apiArg value="12ec30"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="166d28"/>
                            <apiArg value="b8"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH2"/>
                            <exInfo value="CachePath"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.621" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1c0"/>
                            <apiArg value="12ec30"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="166d28"/>
                            <apiArg value="b8"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH3"/>
                            <exInfo value="CachePath"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.622" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1c4"/>
                            <apiArg value="12ec30"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="166d28"/>
                            <apiArg value="b8"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH4"/>
                            <exInfo value="CachePath"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.623" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1b8"/>
                            <apiArg value="12ed4c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12edc4"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH1"/>
                            <exInfo value="CacheLimit"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.624" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1bc"/>
                            <apiArg value="12ed4c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12edc4"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH2"/>
                            <exInfo value="CacheLimit"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.625" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1c0"/>
                            <apiArg value="12ed4c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12edc4"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH3"/>
                            <exInfo value="CacheLimit"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.626" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1c4"/>
                            <apiArg value="12ed4c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12edc4"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_LOCAL_MACHINE\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\CACHE\\PATHS\\PATH4"/>
                            <exInfo value="CacheLimit"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.640" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1b0"/>
                            <apiArg value="12ea98"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="12eb68"/>
                            <apiArg value="52"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\EXPLORER\\SHELL FOLDERS"/>
                            <exInfo value="Cookies"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.650" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1b0"/>
                            <apiArg value="12ea98"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="12eb68"/>
                            <apiArg value="70"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\EXPLORER\\SHELL FOLDERS"/>
                            <exInfo value="History"/>
                        </exInfo_list>
                    </action>
                    <action api_name="Fake_DetectDebugger" call_name="1.exe" call_pid="396" call_time="18:45:41.666" ret_value="1">
                        <apiArg_list count="4">
                            <apiArg value="7ffdfc00"/>
                            <apiArg value="12f280"/>
                            <apiArg value="12f278"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="DiskOperation"/>
                        </exInfo_list>
                    </action>
                    <action api_name="Fake_DetectDebugger" call_name="1.exe" call_pid="396" call_time="18:45:41.679" ret_value="1">
                        <apiArg_list count="4">
                            <apiArg value="7ffdfc00"/>
                            <apiArg value="12f280"/>
                            <apiArg value="12f278"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="DiskOperation"/>
                        </exInfo_list>
                    </action>
                    <action api_name="Fake_DetectDebugger" call_name="1.exe" call_pid="396" call_time="18:45:41.692" ret_value="1">
                        <apiArg_list count="4">
                            <apiArg value="7ffdfc00"/>
                            <apiArg value="12f280"/>
                            <apiArg value="12f278"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="DiskOperation"/>
                        </exInfo_list>
                    </action>
                    <action api_name="Fake_DetectDebugger" call_name="1.exe" call_pid="396" call_time="18:45:41.715" ret_value="abadbeef">
                        <apiArg_list count="3">
                            <apiArg value="194"/>
                            <apiArg value="0"/>
                            <apiArg value="12ed9c"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="sleep(60s)"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtQueryAttributesFile" call_name="1.exe" call_pid="396" call_time="18:45:41.788" ret_value="c0000034">
                        <apiArg_list count="2">
                            <apiArg value="12eb00"/>
                            <apiArg value="12ead8"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\SXS.DLL"/>
                        </exInfo_list>
                    </action>
                    <action api_name="LoadLibraryExW" call_name="1.exe" call_pid="396" call_time="18:45:41.791" ret_value="0">
                        <apiArg_list count="3">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\sxs.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtReadFile" call_name="1.exe" call_pid="396" call_time="18:45:41.806" ret_value="0">
                        <apiArg_list count="9">
                            <apiArg value="1ec"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="12e9f8"/>
                            <apiArg value="12ead0"/>
                            <apiArg value="40"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\WINDOWS\\system32\\shdocvw.dll"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.869" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="208"/>
                            <apiArg value="12f12c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12f62c"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\ZONEMAP"/>
                            <exInfo value="ProxyBypass"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.870" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="208"/>
                            <apiArg value="12f12c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12f62c"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\ZONEMAP"/>
                            <exInfo value="IntranetName"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.871" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="208"/>
                            <apiArg value="12f12c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12f62c"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\ZONEMAP"/>
                            <exInfo value="UNCAsIntranet"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.885" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="208"/>
                            <apiArg value="12f12c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12f62c"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\ZONEMAP"/>
                            <exInfo value="ProxyBypass"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.886" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="208"/>
                            <apiArg value="12f12c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12f62c"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\ZONEMAP"/>
                            <exInfo value="IntranetName"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.887" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="208"/>
                            <apiArg value="12f12c"/>
                            <apiArg value="0"/>
                            <apiArg value="4"/>
                            <apiArg value="12f62c"/>
                            <apiArg value="4"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\ZONEMAP"/>
                            <exInfo value="UNCAsIntranet"/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtSetValueKey" call_name="1.exe" call_pid="396" call_time="18:45:41.904" ret_value="0">
                        <apiArg_list count="6">
                            <apiArg value="1fc"/>
                            <apiArg value="12fd24"/>
                            <apiArg value="0"/>
                            <apiArg value="1"/>
                            <apiArg value="170b90"/>
                            <apiArg value="2"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\USER AGENT\\POST PLATFORM"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="GetForegroundWindow" call_name="1.exe" call_pid="396" call_time="18:45:41.025" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="NtReadFile" call_name="1.exe" call_pid="396" call_time="18:45:41.086" ret_value="0">
                        <apiArg_list count="9">
                            <apiArg value="98"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                            <apiArg value="12fe7c"/>
                            <apiArg value="12fef8"/>
                            <apiArg value="10"/>
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="C:\\program\\1.exe"/>
                        </exInfo_list>
                    </action>
                    <action api_name="Fake_DetectDebugger" call_name="1.exe" call_pid="396" call_time="18:45:42.087" ret_value="abadbeef">
                        <apiArg_list count="3">
                            <apiArg value="150"/>
                            <apiArg value="0"/>
                            <apiArg value="12cff5c"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="sleep(30s)"/>
                        </exInfo_list>
                    </action>
                    <action api_name="GetForegroundWindow" call_name="1.exe" call_pid="396" call_time="18:45:42.088" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:42.125" ret_value="40000000">
                        <apiArg_list count="4">
                            <apiArg value="12f6a8"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12f688"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="MSCTF.Shared.MUTEX.EDF"/>
                        </exInfo_list>
                    </action>
                    <action api_name="GetForegroundWindow" call_name="1.exe" call_pid="396" call_time="18:45:43.220" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="NtCreateMutant" call_name="1.exe" call_pid="396" call_time="18:45:43.247" ret_value="0">
                        <apiArg_list count="4">
                            <apiArg value="12f99c"/>
                            <apiArg value="1f0001"/>
                            <apiArg value="12f97c"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="1">
                            <exInfo value="MSCTF.Shared.MUTEX.EJB"/>
                        </exInfo_list>
                    </action>
                    <action api_name="GetForegroundWindow" call_name="1.exe" call_pid="396" call_time="18:45:43.271" ret_value="30152">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                    <action api_name="NtDeleteValueKey" call_name="1.exe" call_pid="396" call_time="18:45:43.284" ret_value="0">
                        <apiArg_list count="2">
                            <apiArg value="214"/>
                            <apiArg value="12fed8"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="HKEY_CURRENT_USER\\S-1-5-21-117609710-838170752-515967899-1003\\SOFTWARE\\MICROSOFT\\WINDOWS\\CURRENTVERSION\\INTERNET SETTINGS\\USER AGENT\\POST PLATFORM"/>
                            <exInfo value=""/>
                        </exInfo_list>
                    </action>
                    <action api_name="NtTerminateProcess" call_name="1.exe" call_pid="396" call_time="18:45:43.402" ret_value="abadbeef">
                        <apiArg_list count="2">
                            <apiArg value="0"/>
                            <apiArg value="0"/>
                        </apiArg_list>
                        <exInfo_list count="2">
                            <exInfo value="1.exe"/>
                            <exInfo value="396"/>
                        </exInfo_list>
                    </action>
                    <action api_name="AnalyzeStop" call_name="" call_pid="0" call_time="18:50:37.444" ret_value="0">
                        <apiArg_list count="0"/>
                        <exInfo_list count="0"/>
                    </action>
                </action_list>
            </start_boot>
        </file>
    </file_list>
</report>"""

In [113]:
[x for x in test.split("\n") if "action " in x and "api_name" not in x]

[]